# SQL in Python - Connecting to and retrieving data from PostgreSQL

Previously, you have learned how to connect to a SQL database by using a SQL client such as DBeaver.
Apart from connecting to databases, DBeaver also allows you to run SQL queries against the database, create new tables and populate them with data as well as retrieving the data.

Populating tables with data that you have locally on your machine usually requires you to save it in a file, like a CSV, and import it using the DBeaver UI.

Often, before you reached the final step of uploading your dataset, you have performed data cleaning procedures to bring your data into shape. This means we would import the data into Python, clean it, export it to a CSV file, import it into DBeaver and upload the data into the database.

This process requires multiple steps and more than one software. Fortunately, we can reduce the steps by connecting to the database from Python directly, eliminating the need for a separate SQL client.

**In this notebook you will see 2 ways to connect to SQL-Databases and export the data to a CSV file**


## Creating a connection to a PostgreSQL database with Python

There are 2 python packages that are the "go-to" when it comes to connecting to SQL-Databases: `psycopg2` and `sqlalchemy` 

First an example with psycopg2:

In [14]:
import pandas as pd
import psycopg2


In order to create a connection to our PostgreSQL database we need the following information:

- host = the address of the machine the database is hosted on
- port = the virtual gate number through which communication will be allowed
- database = the name of the database
- user = the name of the user
- password = the password of the user

Because we don't want that the database information is published on github we put it into a .env file which is added into the gitignore. 
In these kind of files you can store information that is not supposed to be published.
With the `dotenv` package you can read the `.env` files and get the variables.
The file was 'force added' to the repo using ```git add -f .env``` command. Please follow instructions inside the file to ensure you have the right credentials inside. 

In [15]:
import os
from dotenv import load_dotenv

load_dotenv()

DATABASE = os.getenv('DATABASE')
USER_DB = os.getenv('USER_DB')
PASSWORD = os.getenv('PASSWORD')
HOST = os.getenv('HOST')
PORT = os.getenv('PORT')

The function from the psycopg2 package to create a connection is called `connect()`.
`connect()` expects the parameters listed above as input in order to connect to the database.

In [16]:
# Create connection object conn
conn = psycopg2.connect(
    database=DATABASE,
    user=USER_DB,
    password=PASSWORD,
    host=HOST,
    port=PORT
)

## Retrieving data from the database

Before we can use our connection to get data, we have to create a cursor. A cursor allows Python code to execute PostgreSQL commmands in a database session.
A cursor has to be created with the `cursor()` method of our connection object conn.

In [17]:
cur = conn.cursor()

Now we can run SQL-Queries with `cur.execute('QUERY')` and then run `cur.fetchall()` to get the data:

In [18]:
cur.execute('SELECT * FROM datasets.kaggle_survey LIMIT 10')
cur.fetchall()

[(7232,
  '35-39',
  'Man',
  'Colombia',
  'Doctoral degree',
  'Student',
  '5-10 years',
  'Python',
  'A cloud computing platform (AWS, Azure, GCP, hosted notebooks, etc)',
  '2-5 times',
  '1-2 years',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  'Basic statistical software (Microsoft Excel, Google Sheets, etc.)'),
 (7233,
  '30-34',
  'Man',
  'United States of America',
  'Master’s degree',
  'Data Engineer',
  '5-10 years',
  'Python',
  'A personal computer or laptop',
  '2-5 times',
  '1-2 years',
  '10,000 or more employees',
  '20+',
  'We have well established ML methods (i.e., models in production for more than 2 years)',
  '100,000-124,999',
  '$100,000 or more ($USD)',
  'PostgresSQL ',
  'Microsoft Power BI',
  'Business intelligence software (Salesforce, Tableau, Spotfire, etc.)'),
 (7234,
  '35-39',
  'Man',
  'Argentina',
  'Bachelor’s degree',
  'Software Engineer',
  '10-20 years',
  'R',
  'A personal computer or laptop',
  'Never',
  'I do not use machine learnin

With `conn.close()` you can close the connection again.

In [19]:
#close the connection
conn.close()

But we want to work with the data. The easiest way is to import the data into pandas dataframes. We can use `pd.read_sql_query` or `pd.read_sql_table` or for convenience `pd.read_sql`.

This function is a convenience wrapper around read_sql_table and read_sql_query (for backward compatibility). It will delegate to the specific function depending on the provided input. A SQL query will be routed to read_sql_query , while a database table name will be routed to read_sql_table . Note that the delegated function might have more specific notes about their functionality not listed here.

In [20]:
# Open connection again because we closed it
conn = psycopg2.connect(
    database=DATABASE,
    user=USER_DB,
    password=PASSWORD,
    host=HOST,
    port=PORT
)

In [21]:
# import the data into a pandas dataframe
query_string = "SELECT * FROM datasets.kaggle_survey LIMIT 10"
df_psycopg = pd.read_sql(query_string, conn)

/var/folders/t5/tjfw6fh92wvd7nqk0dkpf15h0000gn/T/ipykernel_94749/3911961467.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_psycopg = pd.read_sql(query_string, conn)


In [22]:
#close the connection
conn.close()

In [23]:
df_psycopg.head()

,id,age_range,gender,county_residence,highest_education,latest_job_role,years_of_programming,programming_language_recommended,computing_platforms,times_tpu_used,years_of_experience,size_of_company,number_of_data_scientists,employer_incorporate_ml,yearly_earnings,money_spend_on_cloud,most_used_data_products,most_used_bi_tool,primary_tool_data_analysis
0,7232,35-39,Man,Colombia,Doctoral degree,Student,5-10 years,Python,"A cloud computing platform (AWS, Azure, GCP, h...",2-5 times,1-2 years,,,,,,,,"Basic statistical software (Microsoft Excel, G..."
1,7233,30-34,Man,United States of America,Master’s degree,Data Engineer,5-10 years,Python,A personal computer or laptop,2-5 times,1-2 years,"10,000 or more employees",20+,"We have well established ML methods (i.e., mod...","100,000-124,999","$100,000 or more ($USD)",PostgresSQL,Microsoft Power BI,"Business intelligence software (Salesforce, Ta..."
2,7234,35-39,Man,Argentina,Bachelor’s degree,Software Engineer,10-20 years,R,A personal computer or laptop,Never,I do not use machine learning methods,"1000-9,999 employees",0,No (we do not use ML methods),"15,000-19,999",$0 ($USD),,,"Basic statistical software (Microsoft Excel, G..."
3,7235,30-34,Man,United States of America,Master’s degree,Data Scientist,5-10 years,Python,"A cloud computing platform (AWS, Azure, GCP, h...",2-5 times,3-4 years,250-999 employees,5-9,"We have well established ML methods (i.e., mod...","125,000-149,999","$10,000-$99,999",MySQL,,"Local development environments (RStudio, Jupyt..."
4,7236,30-34,Man,Japan,Master’s degree,Software Engineer,3-5 years,Python,,,,,,,,,,,


#### SQLALCHEMY

`sqlalchemy` works similarly. Here you have to create an engine with the database sting (a link that includes every information we entered in the conn object)

In [24]:
from sqlalchemy import create_engine

#read the database string from the .env
load_dotenv()

DB_STRING = os.getenv('DB_STRING')

db = create_engine(DB_STRING)

And then you can import that engine with a query into a pandas dataframe.

In [25]:
import pandas as pd

In [26]:
#import the data to a pandas dataframe
query_string = "SELECT * FROM datasets.kaggle_survey"
df_sqlalchemy = pd.read_sql(query_string, db)

In [27]:
df_sqlalchemy.head()

,id,age_range,gender,county_residence,highest_education,latest_job_role,years_of_programming,programming_language_recommended,computing_platforms,times_tpu_used,years_of_experience,size_of_company,number_of_data_scientists,employer_incorporate_ml,yearly_earnings,money_spend_on_cloud,most_used_data_products,most_used_bi_tool,primary_tool_data_analysis
0,7232,35-39,Man,Colombia,Doctoral degree,Student,5-10 years,Python,"A cloud computing platform (AWS, Azure, GCP, h...",2-5 times,1-2 years,,,,,,,,"Basic statistical software (Microsoft Excel, G..."
1,7233,30-34,Man,United States of America,Master’s degree,Data Engineer,5-10 years,Python,A personal computer or laptop,2-5 times,1-2 years,"10,000 or more employees",20+,"We have well established ML methods (i.e., mod...","100,000-124,999","$100,000 or more ($USD)",PostgresSQL,Microsoft Power BI,"Business intelligence software (Salesforce, Ta..."
2,7234,35-39,Man,Argentina,Bachelor’s degree,Software Engineer,10-20 years,R,A personal computer or laptop,Never,I do not use machine learning methods,"1000-9,999 employees",0,No (we do not use ML methods),"15,000-19,999",$0 ($USD),,,"Basic statistical software (Microsoft Excel, G..."
3,7235,30-34,Man,United States of America,Master’s degree,Data Scientist,5-10 years,Python,"A cloud computing platform (AWS, Azure, GCP, h...",2-5 times,3-4 years,250-999 employees,5-9,"We have well established ML methods (i.e., mod...","125,000-149,999","$10,000-$99,999",MySQL,,"Local development environments (RStudio, Jupyt..."
4,7236,30-34,Man,Japan,Master’s degree,Software Engineer,3-5 years,Python,,,,,,,,,,,


Because we don't want to run the queries over and over again we can then export the data into a csv and import that file into our main jupyter notebook: [Visualisation_Exercise](https://github.com/neuefische/ds-visualisation/blob/main/2_Visualisation_Exercise.ipynb)

In [28]:
#export the data to a csv-file
df_sqlalchemy.to_csv('kaggle_survey.csv',index=False)